### Navigation
1. [Day 11](#Day-11)  
1. [Day 12](#Day-12)  
1. [Day 13](#Day-13)  
1. [Day 14](#Day-14)  
1. [Day 15](#Day-15)  

# Day 11
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/11

### --- Part One ---

In [1]:
import numpy as np

with open("data/day11-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]
    inputs = np.asarray([[int(c) for c in r] for r in inputs])
    inputs = np.pad(inputs, [(1,1),(1,1)], constant_values=11)

In [2]:
inputs

array([[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
       [11,  1,  2,  2,  4,  3,  4,  6,  3,  8,  4, 11],
       [11,  5,  6,  2,  1,  1,  2,  8,  5,  8,  7, 11],
       [11,  6,  3,  8,  8,  4,  2,  6,  5,  4,  6, 11],
       [11,  1,  5,  5,  6,  2,  4,  7,  7,  5,  6, 11],
       [11,  1,  4,  5,  1,  8,  1,  1,  5,  7,  3, 11],
       [11,  1,  8,  3,  2,  3,  8,  8,  1,  2,  2, 11],
       [11,  2,  7,  4,  8,  5,  4,  5,  6,  4,  7, 11],
       [11,  2,  5,  8,  2,  8,  7,  7,  4,  3,  2, 11],
       [11,  3,  1,  8,  5,  6,  4,  3,  8,  7,  1, 11],
       [11,  2,  2,  2,  4,  8,  7,  6,  6,  2,  7, 11],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]])

In [3]:
def recursion(inputs, zero_points=[]):
    # get all with power 10
    power10 = np.where(inputs==10)
    power10_pts = list(zip(power10[0], power10[1]))
    # move all the 10s to 11s so that we dont count them anymore
    for pt in power10_pts:
        inputs[pt] += 1
    zero_points += power10_pts
    for r, c in power10_pts:
        inputs[r-1:r+2, c-1:c+2] += 1
        inputs, zero_points = recursion(inputs, zero_points)
    return inputs, zero_points

In [4]:
flash_counter = 0
i = 0
while(True):
    i += 1
    # increase power by 1
    inputs += 1
    inputs, zero_points = recursion(inputs, zero_points=[])
    # increase flash counter
    flash_counter += len(zero_points)
    # replace flashed points with zeros
    for pt in zero_points:
        inputs[pt] = 0
    if i == 100:
        print("Answer for part one:", flash_counter)
    if len(zero_points) == 100:
        print("Answer for part two:", i)
        break
    

Answer for part one: 1591
Answer for part two: 314


# Day 12
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/12

### --- Part One ---

In [5]:
with open("data/day12-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [6]:
from collections import Counter

def add_to_graph(graph, a, b):
    if a in graph.keys():
        graph[a] += [b]
    else:
        graph[a] = [b]
    return graph

def check_revisit_possible(path):
    path = [cave for cave in path
            if str.islower(cave) and cave not in ['start', 'end']]
    counts = Counter(path)
    if sum(counts.values()) == len(counts.keys()):
        return True
    return False

def count_paths(curr_paths, graph, paths_counter, part_two=True):
    for curr_path in curr_paths:
        last_position = curr_path[-1]
        posible_moves = graph[last_position]
        if part_two:
            revisit = check_revisit_possible(curr_path)
        for pos_move in posible_moves:
            if pos_move == 'end':
                paths_counter += 1
            elif (str.isupper(pos_move) or pos_move not in curr_path
                  or (part_two
                      and pos_move != 'start'
                      and revisit)):
                # recursion
                paths_counter = count_paths(
                    [curr_path + [pos_move]], graph, paths_counter, part_two)
    return paths_counter

In [7]:
graph = {}
for mapping in inputs:
    a, b = mapping.split('-')
    add_to_graph(graph, a, b)
    add_to_graph(graph, b, a)

In [8]:
%%time
all_paths = count_paths([['start']], graph, 0, False)
all_paths

Wall time: 24.9 ms


3802

### --- Part Two ---

In [9]:
%%time
all_paths = count_paths([['start']], graph, 0, True)
all_paths

Wall time: 1.73 s


99448

# Day 13
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/13

### --- Part One ---

In [10]:
with open("data/day13-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines() if line != '\n']
    dots = [[int(v) for v in line.split(',')] for line in inputs if ',' in line]
    instructions = [line.split('=') for line in inputs if '=' in line]

In [11]:
import numpy as np

dots_np = np.asarray(dots)

x_max = np.asarray(dots_np)[:,0].max() + 1
y_max = np.asarray(dots_np)[:,1].max() + 1

x_max = x_max + 1 if x_max % 2 == 0 else x_max
y_max = y_max + 1 if y_max % 2 == 0 else y_max

dots_paper_np = np.zeros((y_max, x_max))
for x, y in dots_np:
    dots_paper_np[y, x] = 1

In [12]:
def fold_paper(paper, instr, v):
    if 'x' in instr:
        paper_a = paper[:,:v]
        paper_b = np.fliplr(paper[:,v+1:])
    elif 'y' in instr:
        paper_a = paper[:v,:]
        paper_b = np.flipud(paper[v+1:,:])
    return np.logical_or(paper_a, paper_b)

In [13]:
folded_paper = dots_paper_np.copy()
for i, instr in enumerate(instructions[:]):
    instr, v = instr
    folded_paper = fold_paper(folded_paper, instr, int(v))
    if i == 0:
        print("Answer part one:",folded_paper.sum())

Answer part one: 737


### --- Part Two ---

In [14]:
for i, v in enumerate(range(4,40,5)):
    print("---Letter ", i+1, "---")
    print(folded_paper[:,v-4:v] * 1)

---Letter  1 ---
[[1 1 1 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 1 1 1]]
---Letter  2 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  3 ---
[[0 0 1 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  4 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  5 ---
[[0 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [1 0 0 1]]
---Letter  6 ---
[[1 1 1 1]
 [1 0 0 0]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]
---Letter  7 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]]
---Letter  8 ---
[[1 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 0]]


# Day 14
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/14

### --- Part One ---

In [15]:
with open("data/day14-input1.txt", 'r') as file:
    inputs_raw = file.read()
    inputs = [line for line in inputs_raw.split('\n') if line != '']
    instructions = [line.replace(' ', '').split('->')
                    for line in inputs if '>' in line]

This is an example of template breakdown I'm trying to implement with my code
```
Template:     NNC
              NN NC
After step 1: NCN NBC
              NC CN NB BC
After step 2: NBC CCN NBB BBC
              NB BC CC CN NB BB BB BC
After step 3: NBB BBC CNC CCN NBB BNB BNB BBC
              NB BB BB BC CN NC CC CN NB BB BN NB BN NB BB BC
```
At the end what I get is char pairs counter dictionary.  
I then break down this pairs counter into final per-char counter.  
Because of how the breakdown is made I end up with duplicate counts.  
Only characters that are not duplicated are the edge chars from the template ('N' and 'C')

In [16]:
def calc_rounds(rounds, template, instructions):
    formula_mapping = dict(instructions)
    # formula_mapping = {'FK': 'O', 'BK': 'B', 'PB': 'N', 'VS': 'P', ...}
    pairs_counter = dict(
        [(k, 0) for k in formula_mapping.keys()]
    )
    # pairs_counter = {'FK': 0, 'BK': 0, 'PB': 0, 'VS': 0, 'OF': 0, ...}
    # slice input template chars into a set of pairs
    for i in range(len(template)-1):
        # and count occurances of each pair
        pairs_counter[template[i:i+2]] += 1
    # iterate rounds
    for i in range(rounds):
        counter_pairs = pairs_counter.copy().items()
        # iterate through keys and values of pairs_counter
        for k, v in counter_pairs:
            # engage only if counter indicates given pair exist in current template
            if v > 0:
                # for given pair (k) find a char to be inserted between k[0] and k[1]
                # ex: k == 'CN' and r == 'B'
                r = formula_mapping[k]
                # build new pairs and copy occurances from k pair
                # ex: k[0] + r == 'CB'
                #     r + k[1] == 'BN'
                pairs_counter[k[0] + r] += v
                pairs_counter[r + k[1]] += v
                # since we broke this pair into new pairs
                # decrease occurances for old pair
                pairs_counter[k] -= v
    # calc score
    return calculate_score(
        formula_mapping,
        pairs_counter,
        edge_chars=[template[0], template[-1]])

def calculate_score(formula_mapping, pairs_counter, edge_chars):
    # formula_mapping = {'FK': 'O', 'BK': 'B', 'PB': 'N', 'VS': 'P', ...}
    # pairs_counter = {'FK': 43, 'BK': 799, 'PB': 163, 'VS': 1, 'OF': 161, ...}
    final_counter = {}
    for c in ''.join(formula_mapping.keys()):
        final_counter[c] = 0
    for c, v in [(c, i[1]) for i in pairs_counter.items()
                 for c in i[0]]:
        final_counter[c] += v
    # final_counter = {'F': 2494, 'K': 6204, 'B': 6774, 'P': 1652, 'V': 1288, ...}
    
    # find characters with max and min number of values
    max_k = max(final_counter, key=final_counter.get)
    min_k = min(final_counter, key=final_counter.get)
    # subtract value for min_k from max_k to create a score
    score = final_counter[max_k] - final_counter[min_k]
    # edge chars from the template where not duplicated
    # so we need to add / substract them from the score respectively
    score = score + 1 if max_k in edge_chars else score
    score = score - 1 if min_k in edge_chars else score
    # floor the final score if needed and return it
    # for the final count values we need to divide each by 2
    # because we made duplicates when breaking down original template into pairs
    return int(score / 2)

In [17]:
%%time
calc_rounds(10, inputs[0], instructions)

Wall time: 1.06 ms


3306

### --- Part Two ---

In [18]:
%%time
calc_rounds(40, inputs[0], instructions)

Wall time: 5.5 ms


3760312702877

# Day 15
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/15

### --- Part One ---

In [19]:
inputs = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

In [20]:
import numpy as np

with open("data/day15-input1.txt", 'r') as file:
    inputs = file.read()

In [21]:
inputs = [line for line in inputs.split('\n')]
inputs = np.asarray([[int(i) for i in line] for line in inputs])

In [22]:
import numpy as np

def min_risk(risk_map, r, c):
    risk_map = np.asarray(risk_map)
    # total risk matrix
    tr = np.zeros_like(risk_map)
    rang = list(range(1, r+1))
 
    # Initialize first row of tc array
    # because it's unaffected by adjacent cells
    for j in rang:
        tr[0, j] = tr[0, j-1] + risk_map[0, j]
    # Initialize first column of total cost(tc) array
    # because it's unaffected by adjacent cells
    for i in rang:
        tr[i, 0] = tr[i-1, 0] + risk_map[i, 0]
 
    # Construct rest of the tr array
    for i in rang:
        for j in rang:
            tr[i, j] = min(tr[i-1, j], tr[i, j-1]) + risk_map[i, j]

    return tr[r,c]

In [23]:
min_risk(inputs, len(inputs)-1, len(inputs)-1)

390

### --- Part Two ---

In [24]:
# construct larger cave

size = len(inputs)
M = np.zeros((size * 5, size * 5))
M[:size, :size] = inputs

for c in range(1, 5):
    M[:size, size*c:size*(c+1)] = M[:size, size*(c-1):size*c] + 1

for c in range(0, 5):
    for r in range(1, 5):
        M[size*r:size*(r+1), size*c:size*(c+1)] = M[size*(r-1):size*r, size*c:size*(c+1)] + 1

# np.where faster than M > 9, 4.40ms vs 6.40ms
M = np.where(M > 9, M - 9, M)
# mask = M > 9
# M[mask] -= 9

In [25]:
from queue import PriorityQueue
from collections import defaultdict
        
def dijkstra(risk_map, start_vertex=(0, 0)):
    r_max, c_max = len(risk_map), len(risk_map[0])
    visited = set()
    D = {}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        visited.add(current_vertex)

        r, c = current_vertex

        neighbors = set()
        if r > 0: neighbors.add((r-1, c))
        if r < r_max-1: neighbors.add((r+1, c))        
        if c > 0: neighbors.add((r, c-1))
        if c < c_max-1: neighbors.add((r, c+1))

        for neighbor in neighbors:
            distance = risk_map[neighbor]
            if neighbor not in visited:
                old_cost = D[neighbor] if neighbor in D.keys() else float('inf')
                new_cost = D[current_vertex] + distance
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor))
                    D[neighbor] = new_cost
    return D

In [26]:
%%time

r_max, c_max = len(M), len(M[0])

D = dijkstra(M)
print(D[(r_max-1, c_max-1)])

2814.0
Wall time: 3.15 s
